<a href="https://colab.research.google.com/github/rajeevak40/mnist_fashion_using_tensorflow/blob/main/CNN_with_optimisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing libraries 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn import preprocessing
%matplotlib inline


In [ ]:
pip install keras.tuner


     |████████████████████████████████| 71kB 6.2MB/s 
  Created wheel for keras.tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=db5b5a4cd59ec3064d3955b56b9150d9a4970d9a9f1b0b621cedd7fda3693452
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=ec78b1e1b9abfbd8e2e6e3e3dcafe33d38890cd1da5ed015cf24b95c9ab1ca8e
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras.tuner terminaltables


In [ ]:
# Loading Data
data= keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test)= data.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
# Scaling data set
# RGB values are in range of 0-255 hence scaling it by deviding it with 255
x_train_scale= x_train/255
x_test_scale = x_test/255

In [ ]:
x_train_scale.shape

(60000, 28, 28)

In [ ]:
x_train_scale= x_train_scale.reshape(len(x_train_scale),28,28,1)
x_test_scale= x_test_scale.reshape(len(x_test_scale),28,28,1)


In [ ]:
x_train_scale.shape

(60000, 28, 28, 1)

In [ ]:
def build_model(hp):
    model = keras.Sequential([
    keras.layers.Conv2D( filters = hp.Int('filter1', min_value=32, max_value= 164, step=16), 
                        kernel_size=hp.Choice('kernal1', values=[3, 5,10]), 
                        activation ='relu', 
                        input_shape=(28,28,1)),
    keras.layers.Conv2D( filters = hp.Int('filter2', min_value=32, max_value= 64, step=8), 
                        kernel_size=hp.Choice('kernal1', values=[3, 5,10]), 
                        activation ='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(units= hp.Int('dense_1', min_value=32, max_value= 128, step=16), activation= 'relu'),
    keras.layers.Dense(units= hp.Int('dense_2', min_value=32, max_value= 64, step=16), activation= 'relu'),
    keras.layers.Dense(10, activation= 'softmax')
    ])
    
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('leraning_rate', values=[1e-2, 1e-3])), 
                  loss='sparse_categorical_crossentropy' , 
                  metrics=['accuracy'])
    
    return model

In [ ]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [ ]:
# Tuner search
tuner_search = RandomSearch(build_model, objective='val_accuracy', 
                            max_trials=5, directory='output', project_name='Minst_Fashion')

In [ ]:
tuner_search.search(x_train_scale, y_train, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 00m 42s]
val_accuracy: 0.7883333563804626

Best val_accuracy So Far: 0.906000018119812
Total elapsed time: 00h 04m 01s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner_search.get_best_models(num_models=1)[0]


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                884768    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 914,490
Trainable params: 914,490
Non-trainable params: 0
__________________________________________________

In [ ]:
model.fit(x_test_scale, y_train, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
282/282 [==============================] - 3s 10ms/step - loss: 2.6141 - accuracy: 0.0952 - val_loss: 2.3027 - val_accuracy: 0.1020
Epoch 5/10
282/282 [==============================] - 2s 8ms/step - loss: 2.3026 - accuracy: 0.1025 - val_loss: 2.3031 - val_accuracy: 0.1020
Epoch 6/10
282/282 [==============================] - 2s 8ms/step - loss: 2.3024 - accuracy: 0.0991 - val_loss: 2.3032 - val_accuracy: 0.0900
Epoch 7/10
282/282 [==============================] - 2s 8ms/step - loss: 2.3024 - accuracy: 0.0977 - val_loss: 2.3032 - val_accuracy: 0.0900
Epoch 8/10
282/282 [==============================] - 2s 8ms/step - loss: 2.3026 - accuracy: 0.0976 - val_loss: 2.3034 - val_accuracy: 0.0900
Epoch 9/10
282/282 [==============================] - 2s 8ms/step - loss: 2.3025 - accuracy: 0.1002 - val_loss: 2.3034 - val_accuracy: 0.0900
Epoch 10/10
282/282 [==============================] - 2s 8ms/step - loss: 2.3024 - accuracy: 0.1021 - val_loss: 2.3032 - val_accuracy: 0.0900
